# LSTM

LSTM-NN for predicting bilateral migration flows.

In [1]:
import torch
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler, FunctionTransformer, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

Using cpu


## Prep

In [3]:
data = pd.read_csv("../data/full.csv")

Restrict to OECD destinations:

In [4]:
oecd = [
    "AUS", "AUT", "BEL", "CAN", "CHL", "CZE", "DNK", 
    "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "ISL", 
    "IRL", "ISR", "ITA", "JPN", "KOR", "LUX", "MEX", 
    "NLD", "NZL", "NOR", "POL", "PRT", "SVK", "SVN", 
    "ESP", "SWE", "CHE", "TUR", "GBR", "USA"
]

data = data[data["destination_iso3"].isin(oecd)].reset_index(drop=True)
data.shape

(167400, 31)

Add dyad indicator:

In [5]:
data["dyad"] = [f"{origin}-{dest}" for origin, dest in zip(data["origin_iso3"], data["destination_iso3"])]
data = data[["dyad"] + [col for col in data.columns if col != "dyad"]] # move to front
data.head()

,dyad,origin_iso3,destination_iso3,year,total,total_linear,total_loess,gdp_growth_origin,gdp_growth_destination,gdp_origin,...,conflict_deaths_destination,pop_growth_origin,pop_growth_destination,unemployment_youth_origin,unemployment_youth_destination,origin_former_colony,common_official_language,common_spoken_language,linguistic_proximity,distance
0,ARG-JPN,ARG,JPN,1990,2657.0,2657.0,2657.000000,-2.467214,4.840929,1.413530e+11,...,NaN,1.456403,0.331783,NaN,NaN,0,0.0,0.0,0.0,18372.04
1,ARG-JPN,ARG,JPN,1991,NaN,2715.6,2729.933857,9.133111,3.523357,1.897200e+11,...,NaN,1.424063,0.392820,11.630,4.474,0,0.0,0.0,0.0,18372.04
2,ARG-JPN,ARG,JPN,1992,NaN,2774.2,2796.510564,7.937292,0.900586,2.287790e+11,...,NaN,1.387435,0.371192,13.787,4.363,0,0.0,0.0,0.0,18372.04
3,ARG-JPN,ARG,JPN,1993,NaN,2832.8,2855.902704,8.206979,-0.459220,2.367420e+11,...,NaN,1.357966,0.324168,22.151,5.114,0,0.0,0.0,0.0,18372.04
4,ARG-JPN,ARG,JPN,1994,NaN,2891.4,2908.074731,5.836201,1.083383,2.574400e+11,...,NaN,1.347024,0.279192,25.735,5.459,0,0.0,0.0,0.0,18372.04


Preprocessing-pipeline:

* Log-transform GDP, population, disaster deaths & conflict deaths
* Normalize target & all features
* RF-Impute missing features (*still testing this one*)

In [6]:
class DataPipeline:

    def __init__(self, data: pd.DataFrame) -> None:
        self.data = data
        self.transformer = FunctionTransformer(self.__log_trans)
        self.scaler = MinMaxScaler()
        #self.imputer = KNNImputer()

        self.logged_data = None
        self.scaled_data = None

    def __log_trans(self, x: np.array) -> np.array:
        return np.log(x + 1)

    def log_transform(self) -> None:
        log_cols = self.data.filter(regex="conflict_deaths|gdp(?!_g)|disaster_deaths|population").columns
        logged = self.transformer.fit_transform(data[log_cols])
        self.data[log_cols] = logged
    
    def normalize(self) -> None:
        to_scale = self.data.select_dtypes(include="number").drop("year", axis=1).columns
        scaled = pd.DataFrame(self.scaler.fit_transform(self.data[to_scale]), columns=to_scale)
        self.data[to_scale] = scaled

    def run_pipeline(self) -> pd.DataFrame:
        self.log_transform()
        self.normalize()
        return self.data

In [7]:
pipeline = DataPipeline(data=data)
test = pipeline.run_pipeline()
test.head()

,dyad,origin_iso3,destination_iso3,year,total,total_linear,total_loess,gdp_growth_origin,gdp_growth_destination,gdp_origin,...,conflict_deaths_destination,pop_growth_origin,pop_growth_destination,unemployment_youth_origin,unemployment_youth_destination,origin_former_colony,common_official_language,common_spoken_language,linguistic_proximity,distance
0,ARG-JPN,ARG,JPN,1990,0.000218,0.000218,0.015504,0.287730,0.569837,0.656461,...,NaN,0.619732,0.338260,NaN,NaN,0.0,0.0,0.0,0.0,0.937821
1,ARG-JPN,ARG,JPN,1991,NaN,0.000223,0.015510,0.341932,0.540971,0.676579,...,NaN,0.619045,0.345364,0.139952,0.033210,0.0,0.0,0.0,0.0,0.937821
2,ARG-JPN,ARG,JPN,1992,NaN,0.000228,0.015516,0.336344,0.483509,0.689376,...,NaN,0.618267,0.342847,0.166584,0.031224,0.0,0.0,0.0,0.0,0.937821
3,ARG-JPN,ARG,JPN,1993,NaN,0.000233,0.015520,0.337604,0.453717,0.691715,...,NaN,0.617641,0.337374,0.269854,0.044662,0.0,0.0,0.0,0.0,0.937821
4,ARG-JPN,ARG,JPN,1994,NaN,0.000238,0.015525,0.326527,0.487514,0.697444,...,NaN,0.617409,0.332139,0.314105,0.050836,0.0,0.0,0.0,0.0,0.937821


*Notes for Imputation strategy:*

* Model-based imputation:
    - `scikit-learn` has `IterativeImputer` which fits models (feature with mis. as outcome, all other as predictors) to impute, *but:* currently in experimental stage, not sure if we should rely on this
    - R has tree/random forest-based imputation (e.g. missForest) which could be a solution (all numeric features & target plus one-hot encoded dyads as predictors?); no implementation in python apparently (but can mimick with `IterativeImputer`)
        * Would also be nice given it should be able to account for more complex, non-linear relationships
    - Could also try KNN-Imputation (crashes on my laptop for some reason)

*See:*

[Paper on RF-Imputation](https://bmcmedresmethodol.biomedcentral.com/articles/10.1186/s12874-020-01080-1) (Some strange sounding findings though)

[RF imputation for medical data](https://watermark.silverchair.com/kwt312.pdf?token=AQECAHi208BE49Ooan9kkhW_Ercy7Dm3ZL_9Cf3qfKAc485ysgAAA0owggNGBgkqhkiG9w0BBwagggM3MIIDMwIBADCCAywGCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMyeAqdKwxtiAv4QpHAgEQgIIC_b8zUvt0xk0fuxdJpzaxdjFsUHHknechnsRP4fa9ptKCF5LptJrDGm4WCSSzgB9_kpUT32K6QMshaiqH4kjhZiA3NgBxK3F4Rdv5X0dj2QzPIJrnX8vZuNy3gHnha1Nhn3Z3pubYNa-0E9MVZYe2M3_kb2dvKSSk9nOhHMrKWDeMQVxJ5p1wEznbvZedWtTizsqTFEpaW6gQI02kaY78e4zsir-yyE45t4hJPAkIf_j5v5OZ8YIDBHgPjbhC30gFK6GEvUoioUZXgBVHXB1IF7tDdKaZYFxqE4OeJNeumy3nWu9DVWpoYTbKfqXns6ULDEl92mLURRs6AD7BY3UrWXgsQ_ePRtoUTRGQU_oki4LJovlwBpn0vFr2bg3jayHiTwKwk27BDqIK-9FCxMlPh2W9-66v1mAk5ceNCkx6quesGMId4xT7sMZku2fH33xx0Xwc0mShkdyrm6-QCKReh5q6BrGbeaf5_1mFJ7IS-hOPIDsw4EWYLldTMYHjqK1nfqeDlHJJ8zl510iajnr4Rx5bzNFHuHiKAWpuEmz4esMGA-QJokGwqxiLks_VOaQ7Pr_5we22TEXawq-4vld5H8hl8evIcS39vgVvwB0_5lM7GHYLBgPzSfqvZaKN5Hc2buqasx49BZUyLhi65m33AsQPCMKSNEWqIv5nVtij4U5-ZBr_JYlWkUHNa0yG4xrob7_YyaXJ2TczSkxKByeACPRJf3x7pQVC0ciorLXGEFzRUuIy_gznMpwYCu6DA6MtlhBhZ6nQ1HEGLWCXFfRnBx0S5hipGNGsRpNEEpAJhi5Uefeiy4qa5arGcd6N9xR4S8TsgGAUoPFqXiVgK2RUsTbmDHSAc6348Xp0KNQIWvwLr1-DpstGo1fv_bBxz4w8K-enWx6KK49m7wqaG1WwU60OAzTeCXh77Cqhacjd9t5Gz6nIvmCZbbifldxGMtP20-5PHqRLEdDtASpAY5u1RGuXCAgkXPyg5NMo7XfVi4ns2t5ONnlVtpc8c9aHJg)

[Review of RF imputation](https://arxiv.org/pdf/1701.05305):

* Performance generally robust, increasing with higher feature correlation
* Can deal with high missingness
* Handles non-random missingness well
* Handles complex, non-linear relationships well

In [8]:
test = pd.get_dummies(test, columns=["dyad"], prefix="dyad_") # crashes when using `dtype`-argument

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
imputer = IterativeImputer(estimator=rf, random_state=42)

to_impute = test.select_dtypes(include=["number", "boolean"]).drop(["year", "total"], axis=1).columns
test = pd.DataFrame(imputer.fit_transform(test[to_impute]))
test.head()